In [1]:
import json 
file = open("account.json","r",encoding = "utf-8") # 取得 account.json 其中的帳密資料
strJson = file.read()
file.close()
dictJson = json.loads(strJson) # 存取在記憶體內
print(dictJson)
print(dictJson['username'])
print(dictJson['password'])

{'username': 'jia.ning_cjn', 'password': 'showlo0729'}
jia.ning_cjn
showlo0729


In [2]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 引入 hashlib 模組
import hashlib

# 引入 regular expression 工具
import re


'''
Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
Mouse Hover Action in Selenium
參考網址：https://www.toolsqa.com/selenium-webdriver/mouse-hover-action/
'''
# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         #最大化視窗
options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截

# 使用 Chrome 的 WebDriver (含 options)
driver = webdriver.Chrome( options = options )

# driver.set_window_size(1200, 960) #視窗大小設定 (寬，高)
# driver.maximize_window() #視窗最大化
# driver.minimize_window() #視窗最小化

url = "https://www.instagram.com/weiteng0710/?hl=zh-tw" # 想爬取的資料來源網址
listData = []  # 放置爬蟲資料
listLink = []  # 放置格子內容超連結
setTmp = set() # 放置圖片連結過濾重複元素

def login():
    
    try:
        driver.get('https://www.instagram.com/') #前往首頁
        #等待互動元素出現
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[name="username"]') ))
        # 輸入帳號密碼
        driver.find_element(By.CSS_SELECTOR, 'input[name="username"]').send_keys(dictJson['username'])
        driver.find_element(By.CSS_SELECTOR, 'input[name="password"]').send_keys(dictJson['password'])
        sleep(3)
        #按下登入
        driver.find_element(By.CSS_SELECTOR, 'button[type="submit"].sqdOP.L3NKy.y3zKF').click()
        sleep(4)

                                                                       
    except TimeoutException:
        print("等待逾時")
        sleep(3)
        driver.quit()
        
def visit():
    driver.get(url)
    
def scroll():
    try:
        #等待元素出現
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.Nnq7C.weEfm")))
        '''
        totalHeight => 瀏覽器內部的高度
        offset => 當前捲動的量(高度)
        count => 計無效滾動次數
        limit => 最大無效滾動次數
        wait_second => 每次滾動後的強制等待時間
        '''
        totalHeight = 0
        offset = 0
        count = 0
        limit = 3
        wait_second = 3 #改秒數可以 加快滾動
    
        while count <= limit: # 在捲動到沒有元素產生前 , 持續捲動
            offset = driver.execute_script( # 每次移動的高度
                'return window.document.documentElement.scrollHeight;'
            )
#             offset += 600 # 怕滾太快 可以慢慢滾

            '''
            或是每次只滾動一點距離，
            以免有些網站會在移動長距離後，
            將先前移動當中的元素隱藏
            offset += 600
            '''
            
            js_code = f'''
                    window.scrollTo({{
                        top:{offset},
                        behavior: 'smooth'
                    }});
                '''
            driver.execute_script(js_code)
        
            sleep(wait_second)
        
            totalHeight = driver.execute_script( # 每次移動的高度
                'return window.document.documentElement.scrollHeight;'
            )
        
            if offset >= totalHeight:
                count += 1
        
            if offset >=600: # 為了實驗功能，捲動超過一定的距離，就結束程式
                break
            
    except TimeoutException:
        print("等待逾時, 關閉瀏覽器......")
        sleep(3)
        driver.quit()
        
def getUrl(): #取得每個項目的 url
    
    a_elms = driver.find_elements(By.CSS_SELECTOR,"div.Nnq7C.weEfm > div.v1Nh3.kIKUG._bz0w > a")
    
    for index, a in enumerate(a_elms): # 逐一檢視元素
        if index >= 3: # 測試功能 , 只選三個連結
            break
        
        aLink = a.get_attribute('href') # 取得圖片連結
        print(f"取得網址: {aLink}") # 空格很重要!!
        
        listLink.append(aLink)
    print(listLink)
    
# 逐個網頁內容進行分析
def parse():
    for aLink in listLink:
        setTmp.clear() # 清空戰存圖片的 set 
        driver.get(aLink)
        
        '''
        取得頁面的 id：
            full_match = re.search(regex, aLink)[0]
            group_1 = re.search(regex, aLink)[1]
        另一種做法：
            matchObj = re.search(regex, aLink)
        範例 1：
            matchObj.group()  : /p/CUpOiDxDOu6/
            matchObj.group(1) : CUpOiDxDOu6
        範例 2: 
            matchObj[0]  : /p/CUpOiDxDOu6/
            matchObj[1]  : CUpOiDxDOu6
        '''
        regex = r"\/p\/([A-Za-z0-9-_]+)"
        pageId = re.search(regex,aLink)[1]
        print(f"網頁的ID:{pageId}") 
        sleep(2)
        
        '''
        判斷是否有「向右」按鈕：
            若有，則代表會有多個 li；
            若無，則代表只有一個 li

        備註：
            由於 find_elements 回傳 list 格式，
            所以可以用 len() 來取得元素長度，
            判斷元素是否存在
        '''
        # 當有右箭頭時 代表頁面中有多張照片
        if len(driver.find_elements(By.CSS_SELECTOR, "button._6CZji")) > 0:# 只有 if 只會按一次按鈕
            while len(driver.find_elements(By.CSS_SELECTOR, "button._6CZji")) > 0: # while 會一直按按鈕
                #按下向右按鈕
                driver.find_element(By.CSS_SELECTOR, "button._6CZji").click()
                
                #將元素裡面的屬性值(例如 src)，視為 value 加入 set 當中
                li_elms = driver.find_elements(By.CSS_SELECTOR, "li.Ckrof")
                
                # 檢視當前現有的 li
                for li in li_elms:
                    if len(li.find_elements(By.CSS_SELECTOR, "img.FFVAD")) > 0:
                        imgSrc = li.find_element(By.CSS_SELECTOR, "img.FFVAD").get_attribute('src') # 取得 img src 連結
                        setTmp.add(imgSrc) # 加入圖片連結到 set 當中
                sleep(1)
 
        else: # 沒有右箭頭可以按表示單張圖片
            if len( driver.find_elements(By.CSS_SELECTOR, "img.FFVAD")) > 0:
                imgSrc = driver.find_element(By.CSS_SELECTOR, "img.FFVAD").get_attribute('src') # 取得 img src 連結
                setTmp.add(imgSrc) # 加入圖片連結到 set 當中
                
                
        # 整合元素資料
        listData.append({
            "id" : pageId,
            "link": aLink,
            "listDL": list(setTmp)
        }) 
#     print(listData)

# 將 list 存成 json
def saveJson():
    fp = open("ig.json", "w", encoding = "utf-8")
    fp.write( json.dumps(listData, ensure_ascii=False, indent=4) )
    fp.close
def close():
    driver.quit()
    
def download():
    fp = open("ig.json", "r", encoding="utf-8")
    strJson = fp.read()
    fp.close()
    
    folderPath = 'igPic'
    if not os.path.exists (folderPath):
        os.makedirs(folderPath)
        
    listResult = json.loads(strJson)
    
    # 將 listData 底下的 dict 找出來
    for dictObj in listResult:     
        
        '''dictObj 底下有個 listDL 屬性，它對應的值是個 list，將 list裡面所有 url 找出來'''
        
        for dl_url in dictObj['listDL']:
            print(f"下載連結: {dl_url}")
            strFileName = re.search(r"https?:\/\/\S+.\/(\S+\.(jpe?g|mp4))",dl_url)[1]
            os.system('curl "{}" -o ./{}/{}'.format(dl_url, folderPath, strFileName))
            
        
if __name__ == "__main__":
#     login()
#     visit()
#     scroll()
#     getUrl()
#     parse()
#     saveJson()
    close()
    download()

下載連結: https://instagram.ftpe8-3.fna.fbcdn.net/v/t51.2885-15/fr/e15/s1080x1080/244214064_1052138028862499_4185358378559195066_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net&_nc_cat=111&_nc_ohc=KESlHf1HV4EAX8GRGK5&edm=AABBvjUBAAAA&ccb=7-4&oh=b473c3635406bee25a694b6f6161b531&oe=6163DB14&_nc_sid=83d603
下載連結: https://instagram.ftpe8-3.fna.fbcdn.net/v/t51.2885-15/fr/e15/s1080x1080/244181994_475444426809563_5860783074343906081_n.jpg?_nc_ht=instagram.ftpe8-3.fna.fbcdn.net&_nc_cat=111&_nc_ohc=Aj4YlSHlcVsAX8ceLWW&edm=AABBvjUBAAAA&ccb=7-4&oh=3a9dec18c34635d9640a762354db9186&oe=6163326D&_nc_sid=83d603
下載連結: https://instagram.ftpe8-2.fna.fbcdn.net/v/t51.2885-15/fr/e15/s1080x1080/244422838_390802495869339_677356217874967154_n.jpg?_nc_ht=instagram.ftpe8-2.fna.fbcdn.net&_nc_cat=103&_nc_ohc=8k_NeVdx8A0AX-CoNTX&edm=AABBvjUBAAAA&ccb=7-4&oh=ea665e16b9af0cc2c6abd24968b29cd4&oe=61630CE1&_nc_sid=83d603
下載連結: https://instagram.ftpe8-1.fna.fbcdn.net/v/t51.2885-15/fr/e15/s1080x1080/244277001_395668635364976_68204